In [1]:
import sys
sys.path.insert(0, "../")
sys.path.insert(0, "../torchdrivesim/")

In [2]:
import os
import json
import pickle
import random
import numpy as np
import gymnasium as gym
from typing import Dict, Any, Tuple, List
import torch
from torch import Tensor
from stable_baselines3 import PPO, SAC, TD3, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


import invertedai
from invertedai.common import AgentState, AgentAttributes, Point, TrafficLightState


import torchdriveenv
from torchdriveenv.gym_env import SingleAgentWrapper, WaypointSuiteEnv
from torchdriveenv.env_utils import load_waypoint_suite_data, load_rl_training_config
from torchdrivesim.kinematic import KinematicBicycle

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.


In [3]:
# import wandb
# runs = wandb.Api().runs(path="iai/sb3-single-agent-runs")

In [4]:
# path = "trained_models/TD3_1709879087"
# name = "TD3_1709879087"
# model_path = f"{path}/model"
# algorithm = name.split("_")[0]

In [5]:
def iai_drive(location: str, 
              agent_states: Tensor, 
              agent_attributes: Tensor, 
              recurrent_states: List, 
              traffic_lights_states: Dict = None,
              waypoint_for_ego: Tuple = None):

    try:
        agent_attributes = [AgentAttributes(length=at[0], width=at[1], rear_axis_offset=at[2]) for at in agent_attributes]
        if waypoint_for_ego is not None:
            agent_attributes[0].waypoint = Point(x=waypoint_for_ego[0], y=waypoint_for_ego[1])
        agent_states = [AgentState(center=Point(x=st[0], y=st[1]), orientation=st[2], speed=st[3]) for st in agent_states]
        seed = random.randint(1, 10000)
        response = invertedai.api.drive(
            location=location, agent_states=agent_states, agent_attributes=agent_attributes,
            recurrent_states=recurrent_states,
            traffic_lights_states=traffic_lights_states,
            random_seed=seed
        )
        agent_states = torch.stack(
            [torch.tensor(st.tolist()) for st in response.agent_states], dim=-2
        )
    except Exception as e:
        raise e
    return agent_states, response.recurrent_states

In [6]:
class ExpertEnv(WaypointSuiteEnv):
    def __init__(self, cfg, data):
        # at most 100 NPCs
#         self.observation_space = gym.spaces.Box(shape=(100, 4), dtype=np.float32)
        super().__init__(cfg=cfg, data=data)
#         self.observation_space = gym.spaces.Text(max_length=10)
        self.observation_space = gym.spaces.Box(shape=(100, 4), low=0, high=255, dtype=np.float32)
        

    def get_obs(self):
        location = f'carla:{":".join(self.locations[self.current_waypoint_suite_idx].split("_"))}'

        agent_states = self.simulator.get_innermost_simulator().get_state()["vehicle"].squeeze().cpu().numpy()
        
        agent_attributes = self.simulator._agent_attributes
        
        recurrent_states = self.simulator._recurrent_states
        
        traffic_lights_states = self.simulator._traffic_light_controller.current_state_with_name

        waypoint_for_ego = self.current_target
        
        obs = {"location": location,
               "agent_states": agent_states,
               "agent_attributes": agent_attributes[0],
               "recurrent_states": recurrent_states[0],
               "traffic_lights_states": traffic_lights_states,
               "waypoint_for_ego": waypoint_for_ego}
        
        with open("obs.pkl", "wb") as f:
            pickle.dump(obs, f)
        
#         return "pickle"
        return np.zeros((100, 4))

In [7]:
class Expert:
    def __init__(self):
        self.kinematic_model = KinematicBicycle()
    
    def predict(self, obs, state=None, episode_start=None, deterministic=None):
        with open("obs.pkl", "rb") as f:
            obs = pickle.load(f)
            
        states, recurrent_states = iai_drive(location=obs["location"], 
                                             agent_states=obs["agent_states"], 
                                             agent_attributes=obs["agent_attributes"], 
                                             recurrent_states=obs["recurrent_states"], 
                                             traffic_lights_states=obs["traffic_lights_states"],
                                             waypoint_for_ego=obs["waypoint_for_ego"])
        
        action = self.kinematic_model.fit_action(future_state=states[0], current_state=Tensor(obs["agent_states"][0]))

        return [action], states

In [8]:
val_data = load_waypoint_suite_data("data/validation_cases.yml")
train_data = load_waypoint_suite_data("data/training_cases.yml")
env_config = load_rl_training_config("env_configs/rl_training.yml").env
# env_config.render_mode = "video"
# env_config.video_fov = 450.0

In [9]:
env_config

EnvConfig(ego_only=False, max_environment_steps=200, use_background_traffic=True, terminated_at_infraction=True, seed=None, simulator=TorchDriveConfig(renderer=RendererConfig(backend='default', render_agent_direction=True, left_handed_coordinates=True, highlight_ego_vehicle=True), single_agent_rendering=False, collision_metric=<CollisionMetric.nograd: 'nograd'>, offroad_threshold=0.5, left_handed_coordinates=True), render_mode='rgb_array', video_filename='rendered_video.mp4', video_res=1024, video_fov=500)

In [10]:
def make_expert_env():
    gym.register('expert-env-v0', 
                 entry_point=lambda args: SingleAgentWrapper(ExpertEnv(cfg=args['cfg'], data=args['data'])))
    env = gym.make('expert-env-v0', args={'cfg': env_config, 'data': train_data})
    env = Monitor(env)  
    return env

In [11]:
def make_expert_val_env():
    gym.register('expert-env-v0', 
                 entry_point=lambda args: SingleAgentWrapper(ExpertEnv(cfg=args['cfg'], data=args['data'])))
    env = gym.make('expert-env-v0', args={'cfg': env_config, 'data': val_data})
    env = Monitor(env)  
    return env

In [12]:
# def make_env():
#     gym.register('torchdrivesim/waypoint-suite-v0', entry_point=lambda args: SingleAgentWrapper(WaypointSuiteEnv(cfg=args)))
#     env = gym.make('torchdrivesim/waypoint-suite-v0', args=waypoint_suite_env_config)
#     env = Monitor(env)
#     return env

# def make_val_env():
#     gym.register('torchdrivesim/waypoint-suite-v0', entry_point=lambda args: SingleAgentWrapper(WaypointSuiteEnv(cfg=args)))
#     env = gym.make('torchdrivesim/waypoint-suite-v0', args=eval_env_config)
#     env = Monitor(env, info_keywords=("offroad", "collision", "traffic_light_violation"))
#     return env

In [13]:
class Evaluation:

    def __init__(self, model, eval_env, eval_n_episodes: int, deterministic=False, save_path=None, tab="eval"):
        super().__init__()
        self.eval_n_episodes = eval_n_episodes
        self.deterministic = deterministic
        self.eval_env = eval_env
        self.model = model
        self.save_path = save_path
        self.tab = tab

    def _calc_metrics(self, locals_: Dict[str, Any], globals_: Dict[str, Any]) -> None:
        """
        Callback passed to the  ``evaluate_policy`` function
        Called after each step
        :param locals_:
        :param globals_:
        """
        info = locals_["info"]
        if "psi_smoothness" not in info:
            return
        self.psi_smoothness_for_single_episode.append(info["psi_smoothness"])
        self.speed_smoothness_for_single_episode.append(info["speed_smoothness"])
        if (info["offroad"] > 0) or (info["collision"] > 0) or (info["traffic_light_violation"] > 0) \
                                 or (info["is_success"]):
            self.episode_num += 1

            if info["offroad"] > 0:
                self.offroad_num += 1
            if info["collision"] > 0:
                self.collision_num += 1
            if info["traffic_light_violation"] > 0:
                self.traffic_light_violation_num += 1
            if info["is_success"]:
                self.success_num += 1
            self.reached_waypoint_nums.append(info["reached_waypoint_num"])
            if len(self.psi_smoothness_for_single_episode) > 0:
                self.psi_smoothness.append(sum(self.psi_smoothness_for_single_episode) / len(self.psi_smoothness_for_single_episode))
            if len(self.speed_smoothness_for_single_episode) > 0:
                self.speed_smoothness.append(sum(self.speed_smoothness_for_single_episode) / len(self.speed_smoothness_for_single_episode))


    def evaluate(self) -> bool:
        self.episode_num = 0
        self.offroad_num = 0
        self.collision_num = 0
        self.traffic_light_violation_num = 0
        self.success_num = 0
        self.reached_waypoint_nums = []
        self.psi_smoothness = []
        self.speed_smoothness = []

        mean_episode_reward = 0
        mean_episode_length = 0
        for i in range(self.eval_n_episodes):
            self.psi_smoothness_for_single_episode = []
            self.speed_smoothness_for_single_episode = []
            episode_rewards, episode_lengths = evaluate_policy(
                self.model,
                self.eval_env,
                n_eval_episodes=1,
                deterministic=self.deterministic,
                return_episode_rewards=True,
                callback=self._calc_metrics,
            )
            mean_episode_reward += sum(episode_rewards) / len(episode_rewards)
            mean_episode_length += sum(episode_lengths) / len(episode_lengths)

        mean_episode_reward /= self.eval_n_episodes
        mean_episode_length /= self.eval_n_episodes

        print(f"mean_episode_reward", mean_episode_reward)
        print(f"mean_episode_length", mean_episode_length)

        print(f"offroad_rate", self.offroad_num / self.eval_n_episodes)
        print(f"collision_rate", self.collision_num / self.eval_n_episodes)
        print(f"traffic_light_violation_rate", self.traffic_light_violation_num / self.eval_n_episodes)
        print(f"success_percentage", self.success_num / self.eval_n_episodes)
        print(f"reached_waypoint_num", sum(self.reached_waypoint_nums) / self.eval_n_episodes)
        print(f"psi_smoothness", sum(self.psi_smoothness) / self.eval_n_episodes)
        print(f"speed_smoothness", sum(self.speed_smoothness) / self.eval_n_episodes)

        if self.save_path is not None:
            metrics = {"mean_episode_reward": mean_episode_reward,
                       "mean_episode_length": mean_episode_length,
                       "offroad_rate": self.offroad_num / self.eval_n_episodes,
                       "collision_rate": self.collision_num / self.eval_n_episodes,
                       "traffic_light_violation_rate": self.traffic_light_violation_num / self.eval_n_episodes,
                       "success_percentage": self.success_num / self.eval_n_episodes,
                       "reached_waypoint_num": sum(self.reached_waypoint_nums) / self.eval_n_episodes,
                       "psi_smoothness": sum(self.psi_smoothness) / self.eval_n_episodes,
                       "speed_smoothness": sum(self.speed_smoothness) / self.eval_n_episodes}
            
            with open(f'{self.save_path}/metrics_{self.tab}.json', 'w') as f:
                json.dump(metrics, f)

In [ ]:
model = Expert()
path = "iai_evaluation"

        
print("for validation: ")
eval_val_env = DummyVecEnv([make_expert_val_env])
# eval_val_env = VecFrameStack(eval_val_env, n_stack=3, channels_order="first")

eval_val = Evaluation(model, eval_val_env, eval_n_episodes=10, deterministic=True, save_path=path, tab="validation")
eval_val.evaluate()


print("for training: ")
eval_train_env = DummyVecEnv([make_expert_env])
# eval_train_env = VecFrameStack(eval_train_env, n_stack=3, channels_order="first")

eval_train = Evaluation(model, eval_train_env, eval_n_episodes=10, deterministic=True, save_path=path, tab="train")
eval_train.evaluate()

INFO:torchdriveenv.gym_env:seed: 2796306508
INFO:torchdriveenv.gym_env:    def get_reward(self):
        x = self.simulator.get_state()[..., 0]
        y = self.simulator.get_state()[..., 1]
        psi = self.simulator.get_state()[..., 2]

        d = math.dist((x, y), (self.last_x, self.last_y)) if (self.last_x is not None) and (self.last_y is not None) else 0
        distance_reward = 1 if d > 0.5 else 0
        psi_reward = (1 - math.cos(psi - self.last_psi)) * (-20.0) if (self.last_psi is not None) else 0
        if self.check_reach_target():
            reach_target_reward = 10
            self.reached_waypoint_num += 1
        else:
            reach_target_reward = 0
        r = torch.zeros_like(x)
        r += reach_target_reward + distance_reward + psi_reward
        return r



for validation: 


/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:228: UserWarning: WARN: Expects `terminated` signal to be a boolean, actual type: <class 'torch.Tensor'>
  logger.warn(
/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.

mean_episode_reward 7.421078
mean_episode_length 65.2
offroad_rate 0.7
collision_rate 0.0
traffic_light_violation_rate 0.0
success_percentage 0.3
reached_waypoint_num 0.3
psi_smoothness 0.11129111062912715
speed_smoothness 0.826675123946087
for training: 


In [ ]:
# waypoint_suite_env_config = load_waypoint_suite_env_config("env_configs/training_config.yml")
# eval_env_config = load_waypoint_suite_env_config("env_configs/new_new_waypoint_suite_env_config.yml")

In [ ]:
# waypoint_suite_env_config.iai_gym.ego_only = True
# waypoint_suite_env_config.iai_gym.use_background_traffic = False
# eval_env_config.iai_gym.ego_only = True
# eval_env_config.iai_gym.use_background_traffic = False

In [ ]:
# single_agent_runs = ["SAC_1709170872", "SAC_1709176155", "SAC_1709206540", "SAC_1709206402"]

# for run in runs:
#     if run.name not in single_agent_runs:
#         continue
#     print(run.name)
#     path = f"trained_models/{run.name}"
#     name = run.name
#     model_path = f"{path}/model"
#     algorithm = name.split("_")[0]
#     if not os.path.exists(path):
#         os.mkdir(path)
#     run.file("model.zip").download(root=path, replace=True)
#     if algorithm == "SAC":
#         model = SAC.load(model_path)
    
#     if algorithm == "PPO":
#         model = PPO.load(model_path)
    
#     if algorithm == "A2C":
#         model = A2C.load(model_path)
        
#     if algorithm == "TD3":
#         model = TD3.load(model_path)
        
#     print("for validation: ")
#     eval_val_env = SubprocVecEnv([make_val_env])
#     eval_val_env = VecFrameStack(eval_val_env, n_stack=3, channels_order="first")
    
#     eval_val = Evaluation(model, eval_val_env, eval_n_episodes=10, deterministic=True, save_path=path, tab="validation")
#     eval_val.evaluate()
    
#     print("for training: ")
#     eval_train_env = SubprocVecEnv([make_env])
#     eval_train_env = VecFrameStack(eval_train_env, n_stack=3, channels_order="first")
    
#     eval_train = Evaluation(model, eval_train_env, eval_n_episodes=10, deterministic=True, save_path=path, tab="train")
#     eval_train.evaluate()